# ![RAG](slides/langchain.png)


# ![RAG](slides/image_rag.png)


# LangChain. Naive RAG


# ![RAG](slides/image_langchain.png)



## Settings

Вам нужно будет установить несколько пакетов и установить ваш OpenAI API ключ как переменную окружения с именем `OPENAI_API_KEY`:

In [ ]:
%pip install -qU langchain langchain-openai langchain-chroma beautifulsoup4
%pip install -qU langchain-community pypdf

In [ ]:
# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv

dotenv.load_dotenv('.env', override=True)

In [ ]:
import os

# Print all environment variables
for key, value in os.environ.items():
    print(f"{key}: {value}")


## Index


# ![RAG](slides/image_vectorstore.png)


### Загрузка документов

### Loader
Воспользуемся PyPDFLoader для загрузки документа:

In [ ]:

from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

loader = PyPDFLoader(
    file_path = "./documents/smirnoff_ai.pdf",
    mode = "page",
    extraction_mode = "plain"
     # headers = None
    # password = None,
    # pages_delimiter = "",
    # extract_images = True,
    # images_parser = RapidOCRBlobParser()
)


Теперь прочитаем каждую страницу документа и сохраним в список:

In [ ]:

pages = []
for page in loader.load():
    pages.append(page) 
       
len(pages)

Давайте посмотрим на первую страницу документа:

In [ ]:
print(f"{pages[0].metadata}\n")
print(pages[0].page_content)

### Splitter

Далее мы разбиваем его на меньшие chunks. Воспользуемся RecursiveCharacterTextSplitter:

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(pages)
len(all_splits)

### Vectorstore and embeddings

Теперь загрузим наши чанки в **векторное хранилище**. Одновременно с этим для каждого чанка мы создадим соответствующий векторный образ (эмбеддинг).

Будем использовать эмбеддинг модель из OpenAI:

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Будем использовать векторное хранилище InMemoryVectorStore (в памяти)

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

vector_store = InMemoryVectorStore.from_documents(
    all_splits,
    embedding=OpenAIEmbeddings(model="openai/text-embedding-3-large"), # text-embedding-3-large
)


## Retrieval

Теперь создадим retriever из нашего vectorstore для поиска по нему:

In [ ]:
# k is the number of chunks to retrieve
retriever = vector_store.as_retriever(search_kwargs={'k': 3})

question = "Вы проводите консультации?"
chunks = retriever.invoke(question)

len(chunks)

In [ ]:
print(f"{chunks[0].metadata}\n")
print(chunks[0].page_content)

Мы видим, что вызов retriever выше возвращает некоторые части документа, которую наш чат-бот может использовать в качестве контекста при ответе на вопросы. И теперь у нас есть retriever, который может возвращать связанные данные из документа!

## Augmented generation

# ![RAG](slides/image_prompt_augmentation.png)


Напишем функцию для объединения чанков в одну строку, чтобы позже использовать ее в prompt для задания контекста

In [ ]:
# Concat chunks into a single string to insert into the prompt
def format_chunks(chunks):
    return "\n\n".join(chunk.page_content for chunk in chunks)

chunks_context = format_chunks(chunks)

### PromptTemplate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

SYSTEM_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the user question.
If you don't find the answer in provided context strictly say 'Я не нашел ответа на ваш вопрос!'.
Use three sentences maximum and keep the answer concise.

Context:
{context}
"""

question_answering_prompt = ChatPromptTemplate([
        ("system", SYSTEM_TEMPLATE),
        ("human", "{question}"),
    ]
)

### Chat (LLM)

Будем использовать модель от OpenAI

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="openai/gpt-oss-20b:free", temperature=0.9)

Попросим LLM ответить на вопрос по найденным чанкам в контексте

In [ ]:
from langchain_core.messages import HumanMessage

llm.invoke(question_answering_prompt.invoke(
    {
        "context": chunks_context,
        "question": question,
    })
)

Выглядит хорошо! Для сравнения попробуем без контекстных документов и сравним результат:

In [ ]:
llm.invoke(question_answering_prompt.invoke(
    {
        "context": "",
        "question": question,
    })
)

Браво! Мы с вами создали первую RAG систему!


## Retrieval chains


Давайте теперь создадим цепочку chain, которая будет принимать вопрос от пользователя и возвращать ответ на основе найденных чанков.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_chunks, "question": RunnablePassthrough()}
    | question_answering_prompt
    | llm
    | StrOutputParser()
)

Посмотрите, как удобно теперь использовать - одна строчка кода и мы получаем ответ на наш вопрос:

In [ ]:
# Question
answer = rag_chain.invoke(question)

answer

Выглядит хорошо!


## Conversation mode


### Conversation chat

Создадим шаблон промпта для учета всей истории диалога

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage

CONVERSATION_SYSTEM_TEMPLATE = """
You are an assistant for question-answering tasks. Answer the user's questions based on the conversation history and below context retrieved for the last question. Answer 'Я не нашел ответа на ваш вопрос!' if you don't find any information in the context. Use three sentences maximum and keep the answer concise.\n\nContext retrieved for the last question:\n\n{context}
"""


conversational_answering_prompt = ChatPromptTemplate(
    [
        ("system", CONVERSATION_SYSTEM_TEMPLATE),
        ("placeholder", "{messages}")
    ]
)

conversational_answering_prompt.invoke(
    {
        "context": "Чанки контекста",
        "messages": [
            HumanMessage(content=question)
        ]
    }
)

Реализуем цепочку ответа на последний заданный вопрос.

In [ ]:
from typing import Dict
from langchain_core.runnables import RunnablePassthrough

def get_last_message_for_retriever_input(params: Dict):
    return params["messages"][-1].content


In [ ]:
last_message_retriever_chain = get_last_message_for_retriever_input | retriever | format_chunks 
last_message_retriever_chain.invoke({"messages": [
            HumanMessage(content=question)
        ]})

In [ ]:
rag_conversation_chain = (
    RunnablePassthrough.assign(
        context=get_last_message_for_retriever_input | retriever | format_chunks
    )
    | conversational_answering_prompt
    | llm
    | StrOutputParser()
)

Протестируем нашу новую диалоговую цепочку с одним сообщением

In [ ]:
print(question)

In [ ]:
# Тестируем диалог с одним сообщением
answer = rag_conversation_chain.invoke({"messages": [
    HumanMessage(content=question)
]})
print("Результат диалога:", answer)


Все прекрасно. Мы получили адекватный ответ на наш вопрос

Теперь зададим уточняющий вопрос "А ещё какие?" (имея ввиду, а какие еще услуги или консультации предоставляются?)

In [ ]:
question2 = "А ещё какие?"

In [ ]:
# Тестируем полный диалог с несколькими сообщениями
dialog_result = rag_conversation_chain.invoke({"messages": [
    HumanMessage(content=question), 
    AIMessage(content=answer),
    HumanMessage(content=question2), # "А ещё какие?"
]})
print("Результат диалога с несколькими сообщениями:", dialog_result)

Видим проблему! Наша цепочка не смогла найти ответ на вопрос.
И действительно сам вопрос "А какие еще?" не несет в себе смысла без понимания истории диалога

### Query transformation

Мы понимаем, что чат-боты взаимодействуют с пользователями в режиме беседы и поэтому должны справляться с уточняющими вопросами.

Давайте посмотрим на чанки, которые мы получили при ответе на вопрос `А ещё что?`:

In [ ]:
result_chunks = retriever.invoke(question2)

print(result_chunks[0].page_content)

Видим, что чанк не содержит информации о других услугах. Значит мы были правы в своем предположении о том, что retriever не нашел нужной информации на запрос "А еще какие?"


Чтобы решить эту проблему, мы можем создать для retriever правильный поисковый запрос на основе всей истории переписки.

Применим технику Query Transformation


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

retrieval_query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Transform last user message to a search query in Russian language according to the whole conversation history above to further retrieve the information relevant to the conversation. Try to thorougly analyze all message to generate the most relevant query. The longer result better than short. Let it be better more abstract than specific. Only respond with the query, nothing else.",
        ),
    ]
)

from langchain_openai import ChatOpenAI

llm_query_transform = ChatOpenAI(model="gpt-4o", temperature=0.4)

retrieval_query_transformation_chain = retrieval_query_transform_prompt | llm_query_transform | StrOutputParser()

Мы создали цепочку для переписывания (трансформции) пользовательского сообщения в поисковый запрос для retriever. Проверим ее:

In [ ]:
retrieval_query_transformation_chain.invoke(
    {
        "messages": [
            HumanMessage(content=question), #Какие услуги предоставляются?
            AIMessage(
                content=answer #Да, мы проводим консалтинг по разработке и внедрению AI.
            ),
            HumanMessage(content=question2), #А ещё какие?
        ],
    }
)

Супер! Мы видим осмысленный запрос вместо абстрактного "А какие еще?"

Давайте теперь создадим цепочку, которая будет использоваться для ответа на вопросы, учитывая историю переписки. Эта цепочка должна уметь отвечать на уточняющие вопросы:

In [ ]:
rag_query_transform_chain = (
    RunnablePassthrough.assign(
        context= retrieval_query_transformation_chain | retriever | format_chunks
    )
    | conversational_answering_prompt
    | llm
    | StrOutputParser()
)

Цепочка создана. Теперь проверим ее на наших нескольких сообщениях:

In [ ]:
rag_query_transform_chain.invoke(
    {
        "messages": [
            HumanMessage(content=question), #Какие услуги предоставляются?
            AIMessage(
                content=answer #Да, мы проводим консалтинг по разработке и внедрению AI.
            ),
            HumanMessage(content=question2), #А ещё какие?
        ]
    }
)

Все! Теперь у нас есть цепочка готовая для внедрения в наш бот!
Ура :)

# Advanced RAG

Практика трансформации запроса, рассмотренная нами, - это всего лишь одна из множества практик и подходов построения RAG-системы в реальной жизни для работы с реальными документами и базами знаний:

# ![RAG](slides/image_advanced.png)